In [76]:
from pyquil.api import QPUConnection, QVMConnection, JobConnection
from pyquil.gates import RX, X, RY, MEASURE, I
from pyquil.quil import Program
import pygsti
from pygsti.construction import std1Q_XYI

import numpy as np

qpu = QPUConnection()
qvm = QVMConnection()

In [81]:
num_qubits = 8
qpu.run_and_measure(Program().inst(RX(-np.pi, 0)), [0])

job gKlJQeCzCZrTGIAv is currently queued at position 22
job gKlJQeCzCZrTGIAv is currently queued at position 16
job gKlJQeCzCZrTGIAv is currently queued at position 11
job gKlJQeCzCZrTGIAv is currently queued at position 6
job gKlJQeCzCZrTGIAv is currently queued at position 1
job gKlJQeCzCZrTGIAv is currently running


[[1]]

TypeError: qubit should be an int or Qubit instance

In [61]:
num_qubits = 8
qpu.run_and_measure(prog, [0], 10)

job cnSxVUjggaIllLvA is currently running


'ERROR: Error: could not parse quil program:\n'

In [45]:
print(Program().inst())

RuntimeError: Error encountered while parsing the quil program at line 1 and column 1
Received an '<EOF>' but was expecting one of [ 'DEFGATE', 'DEFCIRCUIT', 'MEASURE', 'LABEL', 'HALT', 'JUMP', 'JUMP-WHEN', 'JUMP-UNLESS', 'RESET', 'WAIT', 'NOP', 'INCLUDE', 'PRAGMA', 'FALSE', 'TRUE', 'NOT', 'AND', 'OR', 'MOVE', 'EXCHANGE', IDENTIFIER ]

In [71]:
from pyquil.api import QPUConnection, QVMConnection
from pyquil.gates import RX, X, RY
from pyquil.quil import Program
import pygsti
from pygsti.construction import std1Q_XYI

import numpy as np

qpu = QPUConnection()
qvm = QVMConnection()

In [72]:
qubit = 4 # Fill this in the the QC you want to analyze!

label_conversion = {"Gx": RX(np.pi/2., qubit), "Gy": RY(np.pi/2., qubit), "Gi": RX(0, qubit)} # Redwood does not currently support identity gates
gs_target = std1Q_XYI.gs_target
preps = std1Q_XYI.fiducials
effects = std1Q_XYI.fiducials
germs = std1Q_XYI.germs

maxLengths = [0, 1, 2, 4, 8, 16] # Adjust this based on your coherence times
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(gs_target.gates.keys(), preps, effects, germs, maxLengths)

In [73]:
ds = pygsti.objects.DataSet(spamLabels=['minus', 'plus']) # These are the labels Sandia has pre-set
for experiment in listOfExperiments:
    prog = Program()
    for gate in experiment:
        prog += Program(label_conversion[gate])
    result = [res[0] for res in qpu.run_and_measure(prog, [qubit], 1000)]
    ds.add_counts_1q(experiment, list(result).count(0), list(result).count(1))
    # We print the number of 0's and 1's for your viewing pleasure.
    print(experiment, list(result).count(0), list(result).count(1))
ds.done_adding_data()

job ZFZSdKdqvfsJDcsj is currently queued at position 21
job ZFZSdKdqvfsJDcsj is currently queued at position 15
job ZFZSdKdqvfsJDcsj is currently queued at position 11
job ZFZSdKdqvfsJDcsj is currently queued at position 6
job ZFZSdKdqvfsJDcsj is currently queued at position 0
{} 0 0
job ayiStpYAUKgldXzD is currently queued at position 21
job ayiStpYAUKgldXzD is currently queued at position 13
job ayiStpYAUKgldXzD is currently queued at position 10
job ayiStpYAUKgldXzD is currently queued at position 5
Gx 601 399
job ZxvxqeCThNNcljqM is currently queued at position 22
job ZxvxqeCThNNcljqM is currently queued at position 18
job ZxvxqeCThNNcljqM is currently queued at position 12
job ZxvxqeCThNNcljqM is currently queued at position 7
job ZxvxqeCThNNcljqM is currently queued at position 3
Gy 593 407
job kmHptWWLFwFhmOUa is currently queued at position 22
job kmHptWWLFwFhmOUa is currently queued at position 16
job kmHptWWLFwFhmOUa is currently queued at position 9
job kmHptWWLFwFhmOUa is c

KeyboardInterrupt: 

In [33]:
results = pygsti.do_long_sequence_gst(ds, gs_target, preps, effects, germs, maxLengths)

--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.24273859394
  1.42215778327
  1.41529400869
  1.40759221072
  0.0472576545548
  0.00849790385145
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.24264068712
  1.41421356237
  1.41421356237
  1.41421356237
  3.68243912484e-16
  1.43359508138e-16
  
--- Iterative MLGST: Iter 1 of 6  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs each, to distribute over 56 params (taken as 1 param groups of ~56 params).
  Sum of Chi^2 = 27.524 (92 data params - 40 model params = expected mean of 52; p-value = 0.997915)
  Completed in 0.2s
  2*Delta(log(L)) = 30.3539
  Iteration 1 took 0.3s
  
--- Iterative MLGST: Iter 2 of 6  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs

In [34]:
print(results.tables['bestGatesetVsTargetTable'])

['Gi', (array([[ -7.77041276e-06]]), None), (0.00079618385134340485, None), (0.0010002999994169883, None)]
['Gx', (array([[ -1.06427154e-06]]), None), (0.00057413573334944598, None), (0.0006395723375687205, None)]
['Gy', (array([[ -4.73832589e-06]]), None), (0.00066503125170437095, None), (0.0008036641593382369, None)]
*** ReportTable object ***
|---------------------------------------------------------------------------------------------------------------------|
|  Gate  |                   Process|Infidelity  |              1/2 Trace|Distance  |               1/2 Diamond-Norm  |
|---------------------------------------------------------------------------------------------------------------------|
|    Gi  |  (array([[ -7.77041276e-06]]), None)  |  (0.00079618385134340485, None)  |  (0.0010002999994169883, None)  |
|---------------------------------------------------------------------------------------------------------------------|
|    Gx  |  (array([[ -1.06427154e-06]]), None)  |  